Import

In [1]:
import cv2
from sort.sort import *
import pandas as pd
from classify_by_movement import *
import pandas as pd
from calculate_features import *

In [2]:
# Show graph inline
%matplotlib inline

## Show results of video

In [24]:
# Open the video file
video_number = str(11)
fps = 50  # Replace with your video's frame rate

#video_path = 'data/data_orig/val/'+video_number+'/'+video_number+'.mp4'
video_path = '../data/VISEM_Tracking/train/11/11.mp4'

## Calculate centroid

In [20]:
# Load the tracking data from a CSV file
df = pd.read_csv('../results/data_sperm_tracking/sperm_tracking_data_'+ video_number +'.csv')

# Calculate velocity for each track_id
df['velocity_x'] = 0.0
df['velocity_y'] = 0.0
df['speed'] = 0.0

# Frame rate of the video (frames per second)
dt = 1 / fps  # Time interval between frames

# Group by track_id and calculate velocity
for track_id, group in df.groupby('track_id'):
    # Calculate displacement (delta x and delta y)
    group['delta_x'] = group['cx'].diff()
    group['delta_y'] = group['cy'].diff()

    # Calculate velocity (pixels per second)
    group['velocity_x'] = group['delta_x'] / dt
    group['velocity_y'] = group['delta_y'] / dt

    # Calculate speed (magnitude of velocity)
    group['speed'] = (group['velocity_x']**2 + group['velocity_y']**2)**0.5
    
    
    # Calculate mean and maximum velocity
    group["mean_velocity"] = group['speed'].mean()
    group["max_velocity"] = group['speed'].max()
    
    df.loc[group.index, ['mean_velocity', 'max_velocity']] = group[['mean_velocity', 'max_velocity']].fillna(0)
    
    # Update the original DataFrame
    df.loc[group.index, ['velocity_x', 'velocity_y', 'speed']] = group[['velocity_x', 'velocity_y', 'speed']].fillna(0)

# Save the updated DataFrame with velocity data
df.to_csv('../results/data_sperm_track_and_velocity/sperm_tracking_with_velocity_' + video_number + '.csv', index=False)

print("Velocity data saved to sperm_tracking_with_velocity.csv")

Velocity data saved to sperm_tracking_with_velocity.csv


## Show tracking and velocitty in video

In [38]:
# Load the tracking data with velocity
df = pd.read_csv('../results/data_sperm_track_and_velocity/sperm_tracking_with_velocity_' + video_number + '.csv')
df_class = pd.read_csv('../results/data_features_labelling/dataset_3c_11.csv')
# Open the video file
cap = cv2.VideoCapture(video_path)
pixel_to_micron = 0.5
trajectories = {}

# Set the slow-motion factor (e.g., 0.5 for half speed, 0.25 for quarter speed)
slow_motion_factor = 0.3 # Adjust this value as needed

# Calculate the new delay between frames
original_delay = int(1000 / fps)  # Delay in milliseconds
new_delay = int(original_delay / slow_motion_factor)

# Process the video frame by frame
frame_id = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Get the data for the current frame
    frame_data = df[df['frame_id'] == frame_id]

    # Draw velocity vectors on the frame
    for _, row in frame_data.iterrows():
        cx, cy = int(row['cx']), int(row['cy'])
        vx, vy = row['velocity_x'], row['velocity_y']

        # Scale the velocity vector for visualization
        scale = 0.3  # Adjust this to make the vectors visible
        end_point = int(cx + vx * scale), int(cy + vy * scale)
        track_id = int(row['track_id'])
        if track_id not in trajectories:
            trajectories[track_id] = []
        trajectories[track_id].append((cx, cy))
        
        n_class = 0

        try:
            n_class = int(df_class[df_class['sperm_id']==track_id]['label'])
        except:
            continue
        
        color = (0, 0, 0)
        
        # Definir el color según la clase
        if n_class == 0:
            color = (0, 255, 0)      # Verde oscuro
        elif n_class == 1:
            color = (255, 0, 0) # Azul
        elif n_class == 2:
            color = (0, 255, 255) # Amarillo
        elif n_class == 3:
            color =  (0, 0, 255)  # Rojo
    
        # Draw path
        for i in range(1, len(trajectories[track_id])):
            cv2.line(frame, (int(trajectories[track_id][i - 1][0]),int(trajectories[track_id][i - 1][1])), (int(trajectories[track_id][i][0]),int(trajectories[track_id][i][1])), color, 1)

        cv2.rectangle(frame, (int(row['xmin']), int(row['ymin'])), (int(row['xmax']), int(row['ymax'])), color, 1)
        
        # Draw the velocity vector
        cv2.putText(frame, str(track_id), (cx + 10, cy), cv2.FONT_HERSHEY_PLAIN, 0.6, (0, 255, 0), 2)


    # Display the frame
    cv2.imshow('Sperm Velocity', frame)

    # Save the frame (optional)
    cv2.imwrite(f'output/frame_{frame_id:04d}.jpg', frame)

    # Wait for the calculated delay
    if cv2.waitKey(new_delay) & 0xFF == ord('q'):
        break
    frame_id += 1

# Release the video capture object and close windows
cap.release()
cv2.destroyAllWindows()

C:\Users\Sergio\AppData\Local\Temp\ipykernel_11904\144952498.py:42: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  n_class = int(df_class[df_class['sperm_id']==track_id]['label'])
